# Imports 

In [1]:
from pyspark.sql import SparkSession

In [80]:
from pyspark.sql import Window

In [2]:
from pyspark.sql import functions as F

# Initialize 

In [3]:
spark = (
    SparkSession.builder.appName("pyspark-notebook")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "512m")
    .config("spark.sql.codegen.wholeStage", False)
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/25 14:07:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc = spark.sparkContext

# Main 

In [5]:
# from schemas import RAW

In [6]:
df = spark.read.json("/data/stage/DMU.json")

In [107]:
window = Window.partitionBy("name").orderBy(F.col("name").desc())

In [108]:
df_filtered = (
    df
    .withColumn("index", F.row_number().over(window))
    .filter(F.col("index") == 1)
    .drop("index")
)

In [110]:
df_filtered = (
    df_filtered
    .select([
        F.col("colorIdentity"),
        F.col("colors"),
        F.col("convertedManaCost"),
        F.col("manaCost"),
        F.col("manaValue"),
        F.col("name"),
        F.col("originalType"),
        F.col("rarity"),
        F.col("subtypes"),
        F.col("supertypes"),
        F.col("text"),
        F.col("toughness"),
        F.col("type"),
        F.col("types"),
        F.col("power"),
    ])
)

In [111]:
df.write.mode("overwrite").parquet("/data/store/DMU.parquet")